# import stuff

In [73]:
import pandas as pd
pd.set_option('max_colwidth', 100)
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix
import itertools
import scipy.stats as st
from scipy import stats
from sklearn.feature_selection import mutual_info_classif
#import seaborn as sns
#from matplotlib import pyplot as plt
#%matplotlib inline
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
import rpy2
#import pingouin as pg
from itertools import combinations
import openpyxl

import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
import rpy2.ipython.html
rpy2.ipython.html.init_printing()
from rpy2.rinterface_lib.embedded import RRuntimeError

In [74]:
#lavaan = importr('lavaan')

In [75]:
# fix from here https://github.com/rpy2/rpy2/issues/882
import os
os.environ['R_HOME'] = '/Users/zeleninam2/miniconda3/envs/env_hitop_MYENVcfa/bin/R' # change this to whatever is relevant to you, or you might not need this line at all.
rbase = importr('base')
utils = importr('utils')

In [76]:
utils = importr('utils')
#quadprog = importr('quadprog')
lavaan = importr('lavaan')
semtools = importr('semTools')
stringr = importr('stringr')
reticulate = importr('reticulate')

In [77]:
# importing stuff the right way
try:
    semTools_version = '.'.join([str(ii) for ii in list(ro.r('packageVersion("semTools")')[0])])
except RRuntimeError:
    semTools_version = ''
# There's a bug in the main semtools libarary, this install from my fork of semtools where I've fixed it.
if semTools_version != '0.5.6.933':
    ro.r("""
        library(devtools)
        install_github("shotgunosine/semTools/semTools@aa68e38f88d9dd881617d5ade429bde2b2a74e08")
        """
    )

# Paths

In [78]:
path_to_item_lookup = '../../data/ValSample/Internalizing-Somatoform Items_DW.xlsx'
path_to_cogmood_questions = '../../data/CFA/cogmood_questions.csv'
path_to_valsample = '../../data/ValSample/inters23.xlsx'
path_to_codesheet = '../../dylan_github/yougov_codesheet_alignment.tsv'
path_to_genpop = '../../data/NIMH0007_genpop_num_OUTPUT.csv'
path_to_enriched = '../../data/NIMH0007_mental_health_num_OUTPUT.csv'

# Functions

In [79]:
def make_hitop_list(data):
    # only hitop cols
    hitop_cols = []
    for c in data.columns:
        if 'hitop' in c and 'today' not in c:
            hitop_cols.append(c)   
    return(hitop_cols)

def make_recontact_list(data):
    list_recontact = []
    for c in data.columns:
        if 'recontact' in c:
            list_recontact.append(c)
    return(list_recontact)

def do_checks(dat, remove_checks):
    assert remove_checks in ['no', 'yes', 'grid', 'gridinitialvisitonly']
    dat['passed_checks'] = True
    dat['passed_grid'] = True
    dat['passed_grid_initialonly'] = True
    dat['passed_list'] = True
    dat.loc[dat.check_moderately != 3, 'passed_checks'] = False
    dat.loc[dat.check_notatall != 1, 'passed_checks'] = False
    dat.loc[dat.check_moderately != 3, 'passed_grid'] = False
    dat.loc[dat.check_notatall != 1, 'passed_grid'] = False
    dat.loc[dat.todaycheck_1 != 1, 'passed_checks'] = False
    dat.loc[dat.todaycheck_2 != 1, 'passed_checks'] = False
    dat.loc[dat.todaycheck_1 != 1, 'passed_list'] = False
    dat.loc[dat.todaycheck_2 != 1, 'passed_list'] = False
    # deal with attention checks - recontact
    dat['passed_checks_recontact'] = True
    dat['passed_grid_recontact'] = True
    dat['passed_list_recontact'] = True
    dat.loc[dat.check_moderately_recontact != 3, 'passed_checks_recontact'] = False
    dat.loc[dat.check_notatall_recontact != 1, 'passed_checks_recontact'] = False
    dat.loc[dat.check_moderately_recontact != 3, 'passed_grid_recontact'] = False
    dat.loc[dat.check_notatall_recontact != 1, 'passed_grid_recontact'] = False
    dat.loc[dat.todaycheck_1_recontact != 1, 'passed_checks_recontact'] = False
    dat.loc[dat.todaycheck_2_recontact != 1, 'passed_checks_recontact'] = False
    dat.loc[dat.todaycheck_1_recontact != 1, 'passed_list_recontact'] = False
    dat.loc[dat.todaycheck_2_recontact != 1, 'passed_list_recontact'] = False
    print('checks_initial:')
    print(1 - dat.loc[:, ['passed_checks', 'passed_grid', 'passed_list']].mean())
    how_many_passed_checks = dat['passed_checks'].value_counts()[True]
    how_many_overall = dat.shape[0]
    print('Passed checks: %d out of %d' % (how_many_passed_checks, how_many_overall))
    print('checks_recontact:')
    print(1 - dat.loc[:, ['passed_checks_recontact', 'passed_grid_recontact', 'passed_list_recontact']].mean()) 
    how_many_passed_checks_recontact = dat['passed_checks_recontact'].value_counts()[True]
    print('Passed checks: %d out of %d' % (how_many_passed_checks_recontact, how_many_overall))
    if remove_checks == 'yes':
        print('removing checks')
        dat = dat.loc[dat['passed_checks'] == True]
        dat = dat.loc[dat['passed_checks_recontact'] == True]
        print(dat.shape)
        print('done removing checks')
    elif remove_checks == 'grid':
        print('removing checks')
        dat = dat.loc[dat['passed_grid'] == True]
        dat = dat.loc[dat['passed_grid_recontact'] == True]
        print(dat.shape)
        print('done removing checks') 
    elif remove_checks == 'gridinitialvisitonly':
        print('removing checks')
        dat = dat.loc[dat['passed_grid'] == True]
        print(dat.shape)
        print('done removing checks')        
    return(dat)

def load_item_lookup():
    # this is a table that aligns scale items with their corresponding questions
    htqs = pd.read_excel(path_to_item_lookup, skiprows=2,
                                names=['id', 'scale', 'item', '_0', '_1', '_2', '_3']).drop(['_0', '_1', '_2', '_3'], axis=1)
    # cmqs is a dataframe with items and responses
    # htcm is the same dataframe, but only displaying HiTOP items
    cmqs = pd.read_csv(path_to_cogmood_questions)
    cmqs = cmqs.rename({'Unnamed: 0': 'id'}, axis=1)
    htcm = cmqs.query("measure == 'HiTOP'")
    # this is a lookup table for subscales, items (sentence questions), and hitop ids
    item_lookup = htcm.loc[:, ['id','subscale', 'item']].merge(htqs.loc[:, ['id','item']], how='inner', on='item', suffixes=['_cm', '_ht'])
    item_lookup['htid'] = 'hitop'+ item_lookup.id_ht.astype(str)
    return(item_lookup)
    
def load_valsample():
    # VALIDATION SAMPLE - main dataframe with responses
    dat_validation = pd.read_excel(path_to_valsample)
    '''resp_cols = responses.columns[responses.columns.isin(item_lookup.htid)]
    # scale scores
    scale_scores = []
    for ss, df in item_lookup.groupby('subscale'):
        tmp = responses.loc[:, df.htid].sum(1)
        tmp.name = ss.replace(" ", "_").replace("/", "_").replace("-", "_")
        scale_scores.append(tmp)
    scale_scores = pd.concat(scale_scores, axis=1)
    scale_names = list(scale_scores.columns.values)'''
    #do_checks(dat_validation)    
    # this is the validation data to be used
    hitop_cols_list = make_hitop_list(dat_validation)
    data_validation = dat_validation[hitop_cols_list]
    data_validation = data_validation.assign(whichdata=['validation']*data_validation.shape[0])
    return(data_validation)

def do_codesheet():
    #loads the codesheet file
    # wherever the emane column exists, creates a dict variablename:ename
    code_book = pd.read_csv(path_to_codesheet, sep='\t')
    ename_lut = {vn:en for vn, en in code_book.loc[code_book.ename.notnull(), ['var_name', 'ename']].values}
    for vn, en in code_book.loc[code_book.ename.notnull(), ['var_name', 'ename']].values:
        ename_lut[vn+'_recontact'] = en+'_recontact'
    tmp = code_book.query('ename.notnull()')
    scale_lut = {}
    for ss, df in tmp.loc[tmp.ename.str.contains('hitop') & ~tmp.ename.str.contains('today')].groupby('subscale'):
        scale_name = ss.replace(" ", "_").replace("/", "_").replace("-", "_")
        items = df.ename.values
        scale_lut[scale_name] = items    
     #   scale_name_initial = scale_name+'_initial'
     #   scale_lut[scale_name_initial] = items 
        scale_name_recontact = scale_name+'_recontact'
        items_recontact = []
        for item in items:
            items_recontact.append(item+'_recontact')
        scale_lut[scale_name_recontact] = np.array(items_recontact, dtype=object)
    # rename checks
    ename_lut['FNM_Q8_5'] ='check_moderately'
    ename_lut['FNM_Q22_3'] ='check_notatall'
    ename_lut['FNM_Q42_m_10'] ='todaycheck_1'
    ename_lut['FNM_44_m_28'] ='todaycheck_2'
    ename_lut['FNM_Q8_5_recontact'] ='check_moderately_recontact'
    ename_lut['FNM_Q22_3_recontact'] ='check_notatall_recontact'
    ename_lut['FNM_Q42_m_10_recontact'] ='todaycheck_1_recontact'
    ename_lut['FNM_44_m_28_recontact'] ='todaycheck_2_recontact'
    return ename_lut, scale_lut
    
def load_data(dataname, doing_checks, doing_remove_checks):
    print(doing_remove_checks)
    assert(dataname in ['genpop','enriched'])
    #codesheet file
    ename_lut, scale_lut = do_codesheet()
    # done working with the codesheet file, now opening the actual data
    if dataname == 'genpop':
        datapath = path_to_genpop
    elif dataname == 'enriched':
        datapath = path_to_enriched
    dat = pd.read_csv(datapath, dtype={'caseid':str}, engine='python')
    #print(dat.head())
    if dataname == 'genpop':
        # ONLY GENPOP drop the .0 that pandas appends for some reason - only for genpop
        dat['caseid'] = dat.caseid.str[:-2]
    dat = dat.rename(ename_lut, axis=1)
    if doing_checks:
        dat = do_checks(dat = dat, remove_checks = doing_remove_checks) 
    hitop_cols_list = make_hitop_list(dat)
    data = dat[hitop_cols_list]
    data = data.assign(whichdata=[dataname]*data.shape[0])
    recontact_cols_list = make_recontact_list(data)
    data_norecontact = data.drop(columns=recontact_cols_list)
    return data, data_norecontact   

def prep_data_recontactanalysis(dataframe_withrecontact):
    df = dataframe_withrecontact
    df_helper = df.copy(deep=True)
    df_rows = df.shape[0]
    df = df.assign(whichvisit=['initialvisit']*df_rows)
    df_helper = df_helper.assign(whichvisit=['recontactvisit']*df_rows)
    # in df, drop all "recontact" columns
    # in df_helper, drop all initial visit columns, then rename the recontact columns
    cols_recontact = []
    cols_original = []
    for c in df.columns:
        if "recontact" in c:
            cols_recontact.append(c)
            cols_original.append(c[:-10])  
    df = df.drop(columns=cols_recontact)
    df_helper = df_helper.drop(columns=cols_original)
    cols_dict = {}
    for cname in cols_recontact:
        cols_dict[cname] = cname[:-10]
    # rename cols in df_helper
    df_helper.rename(columns=cols_dict, inplace=True)    
    mydata_withrecontact = pd.concat([df, df_helper])
    return(mydata_withrecontact)

def print_summary(obj):
    return rprint(summary(obj))

def print_short_summary(invariance_output):
    ro.r("myoutput <- capture.output(summary(invariance_output))") 
    ro.r("stringid <- grep(\"chisq\", myoutput)")
    ro.r("print(myoutput[stringid-1])")
    ro.r("print(myoutput[stringid])")
    return (0)

def print_problematic_items(invariance_output):
    ro.r("myoutput <- capture.output(summary(invariance_output))") 
    print('Problematic items:')
    ro.r("stringidsig <- grep(\"may differ between Groups\", myoutput)")
    ro.r("print(myoutput[stringidsig])")   
    return (0)
    
def extract_p(invariance_output):
    ro.globalenv['invariance_output'] = invariance_output
    ro.r("myoutput <- capture.output(summary(invariance_output))") 
    ro.r("stringid <- grep(\"chisq\", myoutput)")
    ro.r('myrline <- myoutput[stringid]')
    mypline = ro.r('myrline')[0]
    my_p = mypline.split()[-1]
    return(my_p)

def check_secondary_criteria(fit_config):
    Criteria_passed = False
    ro.r("myoutputconfig <- capture.output(summary(fit_config, fit.measures=TRUE))")
    ro.r("stringid_cfi <- grep(\"Robust.*CFI\", myoutputconfig)")
    ro.r("cfi_line <- myoutputconfig[stringid_cfi]")
    cfiline = ro.r('cfi_line')[0]    
    my_cfi = cfiline.split()[-1] 
    my_cfi = float(my_cfi)
    ro.r("stringid_tli <- grep(\"Robust.*TLI\", myoutputconfig)")
    ro.r("tli_line <- myoutputconfig[stringid_tli]")
    tliline = ro.r('tli_line')[0]    
    my_tli = tliline.split()[-1] 
    my_tli = float(my_tli)
    ro.r("stringid_rmsea <- grep(\"Robust.*RMSEA\", myoutputconfig)")
    ro.r("rmsea_line <- myoutputconfig[stringid_rmsea]")
    myrmsealine = ro.r('rmsea_line')[0]
    my_rmsea = myrmsealine.split('\n')[0].split()[-1]
    my_rmsea = float(my_rmsea)   
    if (my_cfi > 0.95) and (my_tli > 0.95) and (my_rmsea < 0.06):
        Criteria_passed = True
    return (Criteria_passed, my_cfi, my_tli, my_rmsea)   

# Run Main Code

## Load all data

### Load item_lookup

In [80]:
item_lookup = load_item_lookup()
item_lookup
#global item_lookup

/Users/zeleninam2/miniconda3/envs/env_NEW01/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


,id_cm,subscale,item,id_ht,htid
0,3.0,anhedonic depression,I had very little energy.,157,hitop157
1,9.0,separation insecurity,I worried that others would abandon me.,81,hitop81
2,16.0,anxious worry,Thoughts were racing through my head.,34,hitop34
3,19.0,well-being,It was easy for me to laugh.,54,hitop54
4,23.0,appetite gain,I stuffed myself with food.,243,hitop243
5,27.0,anhedonic depression,I was unable to enjoy things like I normally do.,182,hitop182
6,28.0,separation insecurity,I wanted other people to take care of me.,69,hitop69
7,41.0,anxious worry,I had a lot of nervous energy.,89,hitop89
8,43.0,separation insecurity,I wanted someone else to make decisions for me.,50,hitop50
9,47.0,social anxiety,I avoided performing or giving a talk in front of others.,129,hitop129


### Load Validation data

In [81]:
data_val = load_valsample()

### Load GenPop data 
### and concatinate Val + Genpop

In [82]:
data_genpop, data_genpop_norecontact = load_data('genpop', doing_checks = True, doing_remove_checks='gridinitialvisitonly')
data_genpop.shape

gridinitialvisitonly
checks_initial:
passed_checks    0.462
passed_grid      0.006
passed_list      0.458
dtype: float64
Passed checks: 269 out of 500
checks_recontact:
passed_checks_recontact    0.548
passed_grid_recontact      0.200
passed_list_recontact      0.548
dtype: float64
Passed checks: 226 out of 500
removing checks
(497, 735)
done removing checks


(497, 167)

In [83]:
data_genpop_norecontact.shape

(497, 84)

In [84]:
data_val.shape

(496, 84)

In [85]:
data_val.head()

,hitop39,hitop77,hitop84,hitop92,hitop93,hitop123,hitop157,hitop182,hitop230,hitop246,hitop20,hitop34,hitop89,hitop203,hitop240,hitop248,hitop265,hitop120,hitop141,hitop243,hitop275,hitop109,hitop280,hitop283,hitop67,hitop142,hitop159,hitop189,hitop5,hitop66,hitop99,hitop181,hitop231,hitop21,hitop90,hitop95,hitop160,hitop254,hitop261,hitop268,hitop15,hitop104,hitop126,hitop211,hitop215,hitop257,hitop40,hitop50,hitop69,hitop81,hitop113,hitop136,hitop151,hitop197,hitop72,hitop140,hitop143,hitop220,hitop16,hitop165,hitop225,hitop247,hitop278,hitop1,hitop17,hitop114,hitop117,hitop124,hitop129,hitop204,hitop222,hitop236,hitop258,hitop9,hitop23,hitop54,hitop106,hitop149,hitop200,hitop244,hitop245,hitop250,hitop281,whichdata
0,4,3,3,3,4,1,3,2,2,2,3,3,2,2,2,1,2,1,2,2,2,2,2,3,3,2,2,2,2,3,2,1,1,3,2,2,1,2,3,1,3,1,3,2,1,2,3,2,2,1,3,2,2,2,3,2,2,2,1,2,1,4,1,3,4,2,3,2,1,3,1,2,1,3,3,3,2,2,2,2,2,1,2,validation
1,2,1,1,2,2,1,2,2,2,1,2,3,2,2,1,1,2,1,1,2,1,1,1,1,2,2,1,2,2,1,1,1,1,2,1,1,2,3,2,2,2,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,2,1,3,1,1,1,1,1,1,1,1,1,1,1,2,3,3,3,3,2,1,3,3,3,validation
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,2,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,4,1,1,1,1,2,1,1,1,1,1,1,3,4,4,4,4,3,4,4,4,4,validation
3,1,1,1,2,1,1,4,1,2,1,3,1,2,3,1,1,2,1,1,1,1,1,1,1,2,2,2,2,1,1,1,1,1,1,1,1,4,4,4,4,1,3,1,1,3,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,1,2,1,1,4,4,1,1,4,3,4,3,3,2,3,3,3,1,validation
4,2,3,4,3,2,3,3,3,1,2,4,3,3,4,3,4,4,1,1,1,1,2,2,1,4,2,4,4,2,1,1,1,1,4,4,4,2,1,2,2,1,1,1,1,1,1,4,2,1,4,4,4,3,1,2,4,4,4,1,1,1,1,3,1,2,1,4,2,1,2,2,2,1,2,2,2,1,2,3,2,2,2,2,validation


In [86]:
mydata_val_genpop = pd.concat([data_val, data_genpop])
mydata_val_genpop.shape

(993, 167)

In [87]:
mydata_val_genpop.head()

,hitop39,hitop77,hitop84,hitop92,hitop93,hitop123,hitop157,hitop182,hitop230,hitop246,hitop20,hitop34,hitop89,hitop203,hitop240,hitop248,hitop265,hitop120,hitop141,hitop243,hitop275,hitop109,hitop280,hitop283,hitop67,hitop142,hitop159,hitop189,hitop5,hitop66,hitop99,hitop181,hitop231,hitop21,hitop90,hitop95,hitop160,hitop254,hitop261,hitop268,hitop15,hitop104,hitop126,hitop211,hitop215,hitop257,hitop40,hitop50,hitop69,hitop81,hitop113,hitop136,hitop151,hitop197,hitop72,hitop140,hitop143,hitop220,hitop16,hitop165,hitop225,hitop247,hitop278,hitop1,hitop17,hitop114,hitop117,hitop124,hitop129,hitop204,hitop222,hitop236,hitop258,hitop9,hitop23,hitop54,hitop106,hitop149,hitop200,hitop244,hitop245,hitop250,hitop281,whichdata,hitop157_recontact,hitop81_recontact,hitop34_recontact,hitop54_recontact,hitop243_recontact,hitop182_recontact,hitop69_recontact,hitop89_recontact,hitop50_recontact,hitop129_recontact,hitop265_recontact,hitop124_recontact,hitop231_recontact,hitop93_recontact,hitop67_recontact,hitop245_recontact,hitop281_recontact,hitop141_recontact,hitop40_recontact,hitop204_recontact,hitop21_recontact,hitop236_recontact,hitop280_recontact,hitop84_recontact,hitop120_recontact,hitop77_recontact,hitop92_recontact,hitop258_recontact,hitop39_recontact,hitop254_recontact,hitop215_recontact,hitop95_recontact,hitop106_recontact,hitop283_recontact,hitop16_recontact,hitop20_recontact,hitop189_recontact,hitop1_recontact,hitop136_recontact,hitop246_recontact,hitop248_recontact,hitop257_recontact,hitop114_recontact,hitop117_recontact,hitop250_recontact,hitop200_recontact,hitop160_recontact,hitop23_recontact,hitop165_recontact,hitop244_recontact,hitop9_recontact,hitop142_recontact,hitop230_recontact,hitop149_recontact,hitop247_recontact,hitop99_recontact,hitop66_recontact,hitop240_recontact,hitop222_recontact,hitop90_recontact,hitop113_recontact,hitop278_recontact,hitop203_recontact,hitop159_recontact,hitop123_recontact,hitop275_recontact,hitop268_recontact,hitop225_recontact,hitop143_recontact,hitop151_recontact,hitop181_recontact,hitop211_recontact,hitop17_recontact,hitop126_recontact,hitop5_recontact,hitop261_recontact,hitop220_recontact,hitop15_recontact,hitop72_recontact,hitop140_recontact,hitop109_recontact,hitop197_recontact,hitop104_recontact
0,4,3,3,3,4,1,3,2,2,2,3,3,2,2,2,1,2,1,2,2,2,2,2,3,3,2,2,2,2,3,2,1,1,3,2,2,1,2,3,1,3,1,3,2,1,2,3,2,2,1,3,2,2,2,3,2,2,2,1,2,1,4,1,3,4,2,3,2,1,3,1,2,1,3,3,3,2,2,2,2,2,1,2,validation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,1,2,2,1,2,2,2,1,2,3,2,2,1,1,2,1,1,2,1,1,1,1,2,2,1,2,2,1,1,1,1,2,1,1,2,3,2,2,2,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,2,1,3,1,1,1,1,1,1,1,1,1,1,1,2,3,3,3,3,2,1,3,3,3,validation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,2,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,4,1,1,1,1,2,1,1,1,1,1,1,3,4,4,4,4,3,4,4,4,4,validation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1,1,2,1,1,4,1,2,1,3,1,2,3,1,1,2,1,1,1,1,1,1,1,2,2,2,2,1,1,1,1,1,1,1,1,4,4,4,4,1,3,1,1,3,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,1,2,1,1,4,4,1,1,4,3,4,3,3,2,3,3,3,1,validation,NaN,NaN,NaN,N

In [88]:
mydata_val_genpop_norecontact = pd.concat([data_val, data_genpop_norecontact])
mydata_val_genpop_norecontact.shape

(993, 84)

### Load Enriched data
### and concatinate Val + enriched

In [89]:
data_enriched, data_enriched_norecontact = load_data('enriched', doing_checks = True, doing_remove_checks='gridinitialvisitonly')
print(data_enriched.shape)

gridinitialvisitonly
checks_initial:
passed_checks    0.293548
passed_grid      0.016129
passed_list      0.290323
dtype: float64
Passed checks: 219 out of 310
checks_recontact:
passed_checks_recontact    0.412903
passed_grid_recontact      0.174194
passed_list_recontact      0.412903
dtype: float64
Passed checks: 182 out of 310
removing checks
(305, 735)
done removing checks
(305, 167)


In [90]:
mydata_val_enriched = pd.concat([data_val, data_enriched])
print(mydata_val_enriched.shape)
mydata_val_enriched_norecontact = pd.concat([data_val, data_enriched_norecontact])
print(mydata_val_enriched_norecontact.shape)

(801, 167)
(801, 84)


In [91]:
mydata_val_enriched.head()

,hitop39,hitop77,hitop84,hitop92,hitop93,hitop123,hitop157,hitop182,hitop230,hitop246,hitop20,hitop34,hitop89,hitop203,hitop240,hitop248,hitop265,hitop120,hitop141,hitop243,hitop275,hitop109,hitop280,hitop283,hitop67,hitop142,hitop159,hitop189,hitop5,hitop66,hitop99,hitop181,hitop231,hitop21,hitop90,hitop95,hitop160,hitop254,hitop261,hitop268,hitop15,hitop104,hitop126,hitop211,hitop215,hitop257,hitop40,hitop50,hitop69,hitop81,hitop113,hitop136,hitop151,hitop197,hitop72,hitop140,hitop143,hitop220,hitop16,hitop165,hitop225,hitop247,hitop278,hitop1,hitop17,hitop114,hitop117,hitop124,hitop129,hitop204,hitop222,hitop236,hitop258,hitop9,hitop23,hitop54,hitop106,hitop149,hitop200,hitop244,hitop245,hitop250,hitop281,whichdata,hitop157_recontact,hitop81_recontact,hitop34_recontact,hitop54_recontact,hitop243_recontact,hitop182_recontact,hitop69_recontact,hitop89_recontact,hitop50_recontact,hitop129_recontact,hitop265_recontact,hitop124_recontact,hitop231_recontact,hitop93_recontact,hitop67_recontact,hitop245_recontact,hitop281_recontact,hitop141_recontact,hitop40_recontact,hitop204_recontact,hitop21_recontact,hitop236_recontact,hitop280_recontact,hitop84_recontact,hitop120_recontact,hitop77_recontact,hitop92_recontact,hitop258_recontact,hitop39_recontact,hitop254_recontact,hitop215_recontact,hitop95_recontact,hitop106_recontact,hitop283_recontact,hitop16_recontact,hitop20_recontact,hitop189_recontact,hitop1_recontact,hitop136_recontact,hitop246_recontact,hitop248_recontact,hitop257_recontact,hitop114_recontact,hitop117_recontact,hitop250_recontact,hitop200_recontact,hitop160_recontact,hitop23_recontact,hitop165_recontact,hitop244_recontact,hitop9_recontact,hitop142_recontact,hitop230_recontact,hitop149_recontact,hitop247_recontact,hitop99_recontact,hitop66_recontact,hitop240_recontact,hitop222_recontact,hitop90_recontact,hitop113_recontact,hitop278_recontact,hitop203_recontact,hitop159_recontact,hitop123_recontact,hitop275_recontact,hitop268_recontact,hitop225_recontact,hitop143_recontact,hitop151_recontact,hitop181_recontact,hitop211_recontact,hitop17_recontact,hitop126_recontact,hitop5_recontact,hitop261_recontact,hitop220_recontact,hitop15_recontact,hitop72_recontact,hitop140_recontact,hitop109_recontact,hitop197_recontact,hitop104_recontact
0,4,3,3,3,4,1,3,2,2,2,3,3,2,2,2,1,2,1,2,2,2,2,2,3,3,2,2,2,2,3,2,1,1,3,2,2,1,2,3,1,3,1,3,2,1,2,3,2,2,1,3,2,2,2,3,2,2,2,1,2,1,4,1,3,4,2,3,2,1,3,1,2,1,3,3,3,2,2,2,2,2,1,2,validation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,1,2,2,1,2,2,2,1,2,3,2,2,1,1,2,1,1,2,1,1,1,1,2,2,1,2,2,1,1,1,1,2,1,1,2,3,2,2,2,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,2,1,3,1,1,1,1,1,1,1,1,1,1,1,2,3,3,3,3,2,1,3,3,3,validation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,2,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,4,1,1,1,1,2,1,1,1,1,1,1,3,4,4,4,4,3,4,4,4,4,validation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1,1,2,1,1,4,1,2,1,3,1,2,3,1,1,2,1,1,1,1,1,1,1,2,2,2,2,1,1,1,1,1,1,1,1,4,4,4,4,1,3,1,1,3,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,1,2,1,1,4,4,1,1,4,3,4,3,3,2,3,3,3,1,validation,NaN,NaN,NaN,N

In [92]:
data_genpop.head()

,hitop157,hitop81,hitop34,hitop54,hitop243,hitop182,hitop69,hitop89,hitop50,hitop129,hitop265,hitop124,hitop231,hitop93,hitop67,hitop245,hitop281,hitop141,hitop40,hitop204,hitop21,hitop236,hitop280,hitop84,hitop120,hitop77,hitop92,hitop258,hitop39,hitop254,hitop215,hitop95,hitop106,hitop283,hitop16,hitop20,hitop189,hitop1,hitop136,hitop246,hitop248,hitop257,hitop114,hitop117,hitop250,hitop200,hitop160,hitop23,hitop165,hitop244,hitop9,hitop142,hitop230,hitop149,hitop247,hitop99,hitop66,hitop240,hitop222,hitop90,hitop113,hitop278,hitop203,hitop159,hitop123,hitop275,hitop268,hitop225,hitop143,hitop151,hitop181,hitop211,hitop17,hitop126,hitop5,hitop261,hitop220,hitop15,hitop72,hitop140,hitop109,hitop197,hitop104,hitop157_recontact,hitop81_recontact,hitop34_recontact,hitop54_recontact,hitop243_recontact,hitop182_recontact,hitop69_recontact,hitop89_recontact,hitop50_recontact,hitop129_recontact,hitop265_recontact,hitop124_recontact,hitop231_recontact,hitop93_recontact,hitop67_recontact,hitop245_recontact,hitop281_recontact,hitop141_recontact,hitop40_recontact,hitop204_recontact,hitop21_recontact,hitop236_recontact,hitop280_recontact,hitop84_recontact,hitop120_recontact,hitop77_recontact,hitop92_recontact,hitop258_recontact,hitop39_recontact,hitop254_recontact,hitop215_recontact,hitop95_recontact,hitop106_recontact,hitop283_recontact,hitop16_recontact,hitop20_recontact,hitop189_recontact,hitop1_recontact,hitop136_recontact,hitop246_recontact,hitop248_recontact,hitop257_recontact,hitop114_recontact,hitop117_recontact,hitop250_recontact,hitop200_recontact,hitop160_recontact,hitop23_recontact,hitop165_recontact,hitop244_recontact,hitop9_recontact,hitop142_recontact,hitop230_recontact,hitop149_recontact,hitop247_recontact,hitop99_recontact,hitop66_recontact,hitop240_recontact,hitop222_recontact,hitop90_recontact,hitop113_recontact,hitop278_recontact,hitop203_recontact,hitop159_recontact,hitop123_recontact,hitop275_recontact,hitop268_recontact,hitop225_recontact,hitop143_recontact,hitop151_recontact,hitop181_recontact,hitop211_recontact,hitop17_recontact,hitop126_recontact,hitop5_recontact,hitop261_recontact,hitop220_recontact,hitop15_recontact,hitop72_recontact,hitop140_recontact,hitop109_recontact,hitop197_recontact,hitop104_recontact,whichdata
0,2,2,3,4,3,2,2,2,1,2,3,1,1,1,2,4,4,3,1,2,2,2,1,1,1,1,1,1,1,1,1,2,3,1,1,2,4,2,2,1,2,1,1,2,4,3,1,4,1,4,4,2,1,4,1,1,1,2,1,2,1,1,4,2,1,2,1,1,2,2,1,1,1,1,1,2,1,1,1,2,1,1,1,3,1,2,4,3,1,2,2,2,2,2,1,1,1,1,4,4,3,1,1,2,2,1,1,1,2,1,1,1,1,1,1,4,1,1,2,2,1,1,1,2,1,1,1,1,3,1,4,1,4,4,2,1,4,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,3,1,1,1,1,1,1,1,genpop
1,2,1,4,4,1,3,1,3,1,2,4,3,1,2,3,4,4,4,2,4,2,3,2,1,2,4,2,2,1,2,1,2,4,1,3,4,2,3,1,2,2,4,2,2,1,4,3,4,4,4,4,2,2,4,4,2,3,2,4,2,2,2,1,3,1,2,1,2,2,2,2,4,2,3,2,2,2,3,1,2,2,1,3,2,1,3,4,1,2,1,2,1,3,4,2,2,2,1,4,4,4,1,3,3,2,2,1,1,4,2,2,2,4,1,2,4,3,4,2,3,2,1,2,2,2,1,2,2,2,3,4,4,4,4,2,2,4,4,3,2,2,4,2,1,1,1,2,1,2,2,2,1,1,2,2,2,2,2,3,1,2,1,1,2,1,2,genpop
2,2,2,3,3,3,2,2,3,2,2,2,4,2,3,2,2,3,2,3,2,2,3,3,3,2,3,2,3,2,2,3,2,4,4,3,2,2,3,1,2,3,2,2,4,3,2,3,2,4,3,2,3,3,2,2,2,2,3,2,3,3,2,3,2,3,3,2,3,2,2,2,3,3,2,2,2,4,3,2,3,2,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,genpop
3,2,1,1,3,2,1,1,1,1,1,1,1,1,1,1,3,1,2,1,1,1,1,1,1,2,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,4,4,3,2,1,3,2,1,1,1,1,1,1,1,1,1,1,1,4,1,1,1,1,1,2,1,1,2,1,2,1,1,1,1,1,2,1,2,4,1,1,1,2,2,1,4,1,1,1,2,2,1,1,1,1,2,1,4,1,1,1,1,1,1,2,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,2,4,2,4,4,2,1,4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,2,1,1,1,2,1,2,4,1,1,genpop
4,4,1,2,4,4,4,4,1,2,3,1,2,1,3,3,2,1,4,3,2,2,3,1,2,4,3,2,2,4,4,2,2,1,1,1,2,3,2,1,2,3,4,2,2,1,3,4,2,1,1,2,4,1,1,2,3,1,1,1,2,1,1,3,3,1,4,2,2,2,3,1,2,2,2,1,4,2,2,2,2,1,1,3,2,1,2,1,3,3,2,1,2,1,2,1,1,3,3,1,2,1,2,2,2,2,2,3,3,3,2,2,3,3,1,3,1,2,1,3,3,2,1,2,3,2,3,2,3,1,3,2,1,1,1,3,2,1,1,

In [93]:
data_genpop_recontactanalysis = prep_data_recontactanalysis(data_genpop)
data_enriched_recontactanalysis = prep_data_recontactanalysis(data_enriched)

In [94]:
mydata_val_genpop_norecontact.head()

,hitop39,hitop77,hitop84,hitop92,hitop93,hitop123,hitop157,hitop182,hitop230,hitop246,hitop20,hitop34,hitop89,hitop203,hitop240,hitop248,hitop265,hitop120,hitop141,hitop243,hitop275,hitop109,hitop280,hitop283,hitop67,hitop142,hitop159,hitop189,hitop5,hitop66,hitop99,hitop181,hitop231,hitop21,hitop90,hitop95,hitop160,hitop254,hitop261,hitop268,hitop15,hitop104,hitop126,hitop211,hitop215,hitop257,hitop40,hitop50,hitop69,hitop81,hitop113,hitop136,hitop151,hitop197,hitop72,hitop140,hitop143,hitop220,hitop16,hitop165,hitop225,hitop247,hitop278,hitop1,hitop17,hitop114,hitop117,hitop124,hitop129,hitop204,hitop222,hitop236,hitop258,hitop9,hitop23,hitop54,hitop106,hitop149,hitop200,hitop244,hitop245,hitop250,hitop281,whichdata
0,4,3,3,3,4,1,3,2,2,2,3,3,2,2,2,1,2,1,2,2,2,2,2,3,3,2,2,2,2,3,2,1,1,3,2,2,1,2,3,1,3,1,3,2,1,2,3,2,2,1,3,2,2,2,3,2,2,2,1,2,1,4,1,3,4,2,3,2,1,3,1,2,1,3,3,3,2,2,2,2,2,1,2,validation
1,2,1,1,2,2,1,2,2,2,1,2,3,2,2,1,1,2,1,1,2,1,1,1,1,2,2,1,2,2,1,1,1,1,2,1,1,2,3,2,2,2,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,2,1,3,1,1,1,1,1,1,1,1,1,1,1,2,3,3,3,3,2,1,3,3,3,validation
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,2,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,4,1,1,1,1,2,1,1,1,1,1,1,3,4,4,4,4,3,4,4,4,4,validation
3,1,1,1,2,1,1,4,1,2,1,3,1,2,3,1,1,2,1,1,1,1,1,1,1,2,2,2,2,1,1,1,1,1,1,1,1,4,4,4,4,1,3,1,1,3,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,1,2,1,1,4,4,1,1,4,3,4,3,3,2,3,3,3,1,validation
4,2,3,4,3,2,3,3,3,1,2,4,3,3,4,3,4,4,1,1,1,1,2,2,1,4,2,4,4,2,1,1,1,1,4,4,4,2,1,2,2,1,1,1,1,1,1,4,2,1,4,4,4,3,1,2,4,4,4,1,1,1,1,3,1,2,1,4,2,1,2,2,2,1,2,2,2,1,2,3,2,2,2,2,validation


### Concatinate Genpop + Enriched

In [95]:
data_genpop_norecontact.head

<bound method NDFrame.head of      hitop157  hitop81  hitop34  hitop54  hitop243  hitop182  hitop69  \
0           2        2        3        4         3         2        2   
1           2        1        4        4         1         3        1   
2           2        2        3        3         3         2        2   
3           2        1        1        3         2         1        1   
4           4        1        2        4         4         4        4   
..        ...      ...      ...      ...       ...       ...      ...   
495         2        2        2        2         2         2        3   
496         1        1        1        1         1         1        1   
497         1        1        1        4         1         1        1   
498         3        1        2        2         2         2        2   
499         2        1        1        4         1         3        1   

     hitop89  hitop50  hitop129  hitop265  hitop124  hitop231  hitop93  \
0          2       

In [96]:
print(data_genpop_norecontact.shape)
print(data_enriched_norecontact.shape)

(497, 84)
(305, 84)


In [97]:
data_genpop_enriched = pd.concat([data_genpop_norecontact, data_enriched_norecontact])
print(data_genpop_enriched.shape)
data_genpop_enriched.whichdata.value_counts()

(802, 84)


whichdata
genpop      497
enriched    305
Name: count, dtype: int64

# Do main CFA analysis

# 3-way CFA

# Bruteforcing 3-way CFA

function for anx worry and well being

In [98]:
def three_way_cfa_bruteforce(whichscale, whichcfa, howmanyitems):
    print('THIS ONE RUNS ALL COMBINATIONS OF X ITEMS and checks if a combination is 3-way invariant')
    assert(whichscale in ['anxious_worry', 'well_being'])
    flag_found_inv_in_all_three = False
    num_iter = 1000
    rprint = ro.r('print')
    summary = ro.r('summary')
    ro.r('set.seed(12345)')
    rngkind = "L'Ecuyer-CMRG"
    mdl_strs = {}
    for ss, df in item_lookup.groupby('subscale'):
        cln_ss = ss.replace(" ", "_").replace("/", "_").replace("-", "_")
        items = []
        for itemn, item in enumerate(df.sort_values('id_ht').htid.values):
            items.append(item)
        mdl_strs[cln_ss] = f'{cln_ss} =~' + ' + '.join(items) 
    mdl_strs_new = {'anxious_worry': 'anxious_worry =~hitop20 + hitop34 + hitop89 + hitop203 + hitop248 + hitop265',
     'well_being': 'well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245 + hitop250 + hitop281'}
    for cln_ss, mdl in mdl_strs_new.items():
        if cln_ss == whichscale:  
            print('lets go')
            if whichcfa == 'metric':
                do_metric = True
                do_scalar = False
                do_strict = False
            elif whichcfa == 'scalar':
                do_metric = True
                do_scalar = True
                do_strict = False   
            elif whichcfa == 'strict':
                do_metric = True
                do_scalar = True
                do_strict = True                  
            print("\n")
            print(cln_ss.upper())
            print(mdl)
            # ====== CONFIG =====        

            # loop to do ablations
            temp_items = mdl_strs_new[cln_ss]
            temp_items_items = temp_items.split("=~",1)[1]
            temp_items_items_items = temp_items_items.split(" + ")
            list_of_items = temp_items_items_items

            for com in combinations(list_of_items, howmanyitems):
                print('\nCOMBINATIONS OF ITEMS: ')
                print(com)
                mynewstring = temp_items.split("=~",1)[0] + "=~"
                for x in com:
                    mynewstring += x + ' + '
                mynewstring = mynewstring[:-3]
                print(mynewstring)
            
                ro.globalenv[cln_ss+'_mdl'] = mynewstring    

                # set flags for metric invariance genpop and enriched to FALSE
                metric_inv_genpop = False
                metric_inv_enriched = False
                metric_inv_genpopvsenriched = False
                
                # FIRST TRY GENPOP

                print('VALID VS GENPOP')
                
                # to load the data in R, let's save it as csv and then load it to R
                # there may be a more elegant day to do this, but I don't know it...
                
                mydata_gp = mydata_val_genpop_norecontact
                mydata_gp.to_csv('../../data/CFA/mydata_valid_genpop.csv')
                ro.r("rdata <- read.csv('../../data/CFA/mydata_valid_genpop.csv', header=TRUE)")
                group = 'whichdata'

                ro.globalenv['group'] = group
            
                fit_config = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV")')
                ro.globalenv['fit_config'] = fit_config        
                out_config = semtools.permuteMeasEq(nPermute=num_iter, # 100 for now, 1000 for the "real" run
                                                        con=fit_config, # In the case of testing configural invariance when modelType = "mgcfa", con is the configural model (implicitly, the unconstrained model is the saturated model, so use the defaults uncon = NULL and param = NULL).
                                                        parallelType="multicore", ncpus=4)
                config_p = extract_p(out_config)
                if float(config_p) >= 0.05:
                    print('CONFIG INVARIANT chisq p = ' + str(config_p))
                else:
                    (passed_flag, my_cfi, my_tli, my_rmsea) = check_secondary_criteria(fit_config)
                    if passed_flag:
                        print('CONFIG chisq p = ' + str(config_p))
                        print('PASSED SECONDARY CRITERIA WITH CFI = ' + str(my_cfi) + ' TLI = ' + str(my_tli) + ' RMSEA = ' + str(my_rmsea))
                    else:
                        print('CONFIG INVARIANT NOT PASSED, chisq p = ' + str(config_p))
                if do_metric:
                    # ====== METRIC =====
                    fit_metric = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal="loadings")')
                    ro.globalenv['fit_metric'] = fit_metric
                    out_metric = semtools.permuteMeasEq(nPermute=num_iter, 
                                                        uncon=fit_config, 
                                                        con=fit_metric, 
                                                        param="loadings",
                                                        parallelType="multicore", ncpus=4)
                    metric_p = extract_p(out_metric)
                    if float(metric_p) >= 0.05:
                        print('METRIC INVARIANT chisq p = ' + str(metric_p))
                        metric_inv_genpop = True
                    else:
                        # significant:
                        print('METRIC INVARIANT NOT PASSED, chisq p = ' + str(metric_p))
                else:
                    print('METRIC N/A')
                    print('SCALAR N/A')
                    print('STRICT N/A')
                    metric_p = 'NA'
                    scalar_p = 'NA'
                    strict_p = 'NA'


                # NOW TRY ENRICHED

                print('VALID VS ENRICHED')

                mydata_enr = mydata_val_enriched_norecontact
                mydata_enr.to_csv('../../data/CFA/mydata_valid_enriched.csv')
                ro.r("rdata <- read.csv('../../data/CFA/mydata_valid_enriched.csv', header=TRUE)") 
                group = 'whichdata'

                ro.globalenv['group'] = group
            
                fit_config = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV")')
                ro.globalenv['fit_config'] = fit_config        
                out_config = semtools.permuteMeasEq(nPermute=num_iter, # 100 for now, 1000 for the "real" run
                                                        con=fit_config, # In the case of testing configural invariance when modelType = "mgcfa", con is the configural model (implicitly, the unconstrained model is the saturated model, so use the defaults uncon = NULL and param = NULL).
                                                        parallelType="multicore", ncpus=4)
                config_p = extract_p(out_config)
                if float(config_p) >= 0.05:
                    print('CONFIG INVARIANT chisq p = ' + str(config_p))
                else:
                    (passed_flag, my_cfi, my_tli, my_rmsea) = check_secondary_criteria(fit_config)
                    if passed_flag:
                        print('CONFIG chisq p = ' + str(config_p))
                        print('PASSED SECONDARY CRITERIA WITH CFI = ' + str(my_cfi) + ' TLI = ' + str(my_tli) + ' RMSEA = ' + str(my_rmsea))
                    else:
                        print('CONFIG INVARIANT NOT PASSED, chisq p = ' + str(config_p))

                if do_metric:
                    # ====== METRIC =====
                    fit_metric = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal="loadings")')
                    ro.globalenv['fit_metric'] = fit_metric
                    out_metric = semtools.permuteMeasEq(nPermute=num_iter, 
                                                        uncon=fit_config, 
                                                        con=fit_metric, 
                                                        param="loadings",
                                                        parallelType="multicore", ncpus=4)
                    metric_p = extract_p(out_metric)
                    if float(metric_p) >= 0.05:
                        print('METRIC INVARIANT chisq p = ' + str(metric_p))
                        metric_inv_enriched = True
                    else:
                        # significant:
                        print('METRIC INVARIANT NOT PASSED, chisq p = ' + str(metric_p))
                else:
                    print('METRIC N/A')
                    print('SCALAR N/A')
                    print('STRICT N/A')
                    metric_p = 'NA'
                    scalar_p = 'NA'
                    strict_p = 'NA'

                
                # NOW TRY ENRICHED VS GENPOP

                print('GENPOP VS ENRICHED')

                mydata_gpvsen = data_genpop_enriched
                mydata_gpvsen.to_csv('../../data/CFA/mydata_genpop_enriched.csv')
                ro.r("rdata <- read.csv('../../data/CFA/mydata_genpop_enriched.csv', header=TRUE)")
                group = 'whichdata' 
                ro.globalenv['group'] = group
                
                fit_config = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV")')
                ro.globalenv['fit_config'] = fit_config        
                out_config = semtools.permuteMeasEq(nPermute=num_iter, # 100 for now, 1000 for the "real" run
                                                        con=fit_config, # In the case of testing configural invariance when modelType = "mgcfa", con is the configural model (implicitly, the unconstrained model is the saturated model, so use the defaults uncon = NULL and param = NULL).
                                                        parallelType="multicore", ncpus=4)
                config_p = extract_p(out_config)
                if float(config_p) >= 0.05:
                    print('CONFIG INVARIANT chisq p = ' + str(config_p))
                else:
                    (passed_flag, my_cfi, my_tli, my_rmsea) = check_secondary_criteria(fit_config)
                    if passed_flag:
                        print('CONFIG chisq p = ' + str(config_p))
                        print('PASSED SECONDARY CRITERIA WITH CFI = ' + str(my_cfi) + ' TLI = ' + str(my_tli) + ' RMSEA = ' + str(my_rmsea))
                    else:
                        print('CONFIG INVARIANT NOT PASSED, chisq p = ' + str(config_p))

                if do_metric:
                    # ====== METRIC =====
                    fit_metric = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal="loadings")')
                    ro.globalenv['fit_metric'] = fit_metric
                    out_metric = semtools.permuteMeasEq(nPermute=num_iter, 
                                                        uncon=fit_config, 
                                                        con=fit_metric, 
                                                        param="loadings",
                                                        parallelType="multicore", ncpus=4)
                    metric_p = extract_p(out_metric)
                    if float(metric_p) >= 0.05:
                        print('METRIC INVARIANT chisq p = ' + str(metric_p))
                        metric_inv_enriched = True
                    else:
                        # significant:
                        print('METRIC INVARIANT NOT PASSED, chisq p = ' + str(metric_p))
                else:
                    print('METRIC N/A')
                    print('SCALAR N/A')
                    print('STRICT N/A')
                    metric_p = 'NA'
                    scalar_p = 'NA'
                    strict_p = 'NA'


                if metric_inv_genpop == True and metric_inv_enriched == True and metric_inv_genpopvsenriched == True:
                    print("\n!!! THIS COMBINATION:")
                    print(com)
                    print("!!! PASSES 3-WAY INVARIANCE")
                    flag_found_inv_in_all_three = True
                        
    print("DONE")
    
    if flag_found_inv_in_all_three == True:
        print('Found at least 1 combination that is 3-way invariant')
    else:
        print('Could not find a combination that is invariant in both datasets')
    return(0)

In [99]:
## running bruteforcing on both anxious worry and well-being in one cell

In [100]:
'''
print ('\n\n-----ANXIOUS WORRY-----\n\n')

print ('\n\nRUNNING ANXIOUS WORRY FOR 4 ITEMS (THATS N-2)\n\n')
three_way_cfa_bruteforce(whichscale = 'anxious_worry', 
                                 whichcfa = 'metric',
                                 howmanyitems = 4)

print ('\n\njust to be safe - queuing ANXIOUS WORRY FOR 3 ITEMS (THATS N-3)\n\n')
three_way_cfa_bruteforce(whichscale = 'anxious_worry', 
                                 whichcfa = 'metric',
                                 howmanyitems = 3)
'''

"\nprint ('\n\n-----ANXIOUS WORRY-----\n\n')\n\nprint ('\n\nRUNNING ANXIOUS WORRY FOR 4 ITEMS (THATS N-2)\n\n')\nthree_way_cfa_bruteforce(whichscale = 'anxious_worry', \n                                 whichcfa = 'metric',\n                                 howmanyitems = 4)\n\nprint ('\n\njust to be safe - queuing ANXIOUS WORRY FOR 3 ITEMS (THATS N-3)\n\n')\nthree_way_cfa_bruteforce(whichscale = 'anxious_worry', \n                                 whichcfa = 'metric',\n                                 howmanyitems = 3)\n"

In [ ]:
print ('\n\n-----WELL-BEING-----\n\n')

'''
print ('\n\nRUNNING WELL BEING FOR 7 ITEMS (THATS N-3)\n\n')
three_way_cfa_bruteforce(whichscale = 'well_being', 
                                 whichcfa = 'metric',
                                 howmanyitems = 7)

print ('\n\njust to be safe - queuing WELL BEING FOR 6 ITEMS (THATS N-4)\n\n')
three_way_cfa_bruteforce(whichscale = 'well_being', 
                                 whichcfa = 'metric',
                                 howmanyitems = 6)
'''

print ('\n\nvery worst case - queuing WELL BEING FOR 5 ITEMS (THATS N-5)\n\n')
three_way_cfa_bruteforce(whichscale = 'well_being', 
                                 whichcfa = 'metric',
                                 howmanyitems = 5)

print ('\n\nvery very worst case - queuing WELL BEING FOR 4 ITEMS (THATS N-6)\n\n')
three_way_cfa_bruteforce(whichscale = 'well_being', 
                                 whichcfa = 'metric',
                                 howmanyitems = 4)

print ('\n\napocalypsis - queuing WELL BEING FOR 3 ITEMS (THATS N-7)\n\n')
three_way_cfa_bruteforce(whichscale = 'well_being', 
                                 whichcfa = 'metric',
                                 howmanyitems = 3)

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.






-----WELL-BEING-----




very worst case - queuing WELL BEING FOR 5 ITEMS (THATS N-5)


THIS ONE RUNS ALL COMBINATIONS OF X ITEMS and checks if a combination is 3-way invariant
lets go


WELL_BEING
well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245 + hitop250 + hitop281

COMBINATIONS OF ITEMS: 
('hitop9', 'hitop23', 'hitop54', 'hitop106', 'hitop149')
well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149
VALID VS GENPOP


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




CONFIG INVARIANT chisq p = 0.94


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




...

METRIC INVARIANT NOT PASSED, chisq p = 0
VALID VS ENRICHED


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




CONFIG INVARIANT chisq p = 0.057


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




METRIC INVARIANT chisq p = 0.071
GENPOP VS ENRICHED


In [52]:
# code to check which items mean what 
# 'well_being': 'well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245 + hitop250 + hitop281'}

#kept 
items_to_check = ['114', '129', '204', '222', '236', '258'] # manually change this
for i in items_to_check:
    a = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['htid']).split(' ', 1)[1]
    aa = str(a).split('\n')[0]
    b = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['item']).split(' ', 1)[1]
    bb = str(b).split('\n')[0]
    print(aa+bb)
print('\n')

#removed
items_to_check = ['1', '17', '117', '124'] #manually chanhe this
for i in items_to_check:
    a = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['htid']).split(' ', 1)[1]
    aa = str(a).split('\n')[0]
    b = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['item']).split(' ', 1)[1]
    bb = str(b).split('\n')[0]
    print(aa+bb)

   hitop114   I had difficulty making eye contact with others.
   hitop129   I avoided performing or giving a talk in front of others.
   hitop204   I felt uncomfortable being the center of attention.
   hitop222   I was uncomfortable entering a room when others already were present.
   hitop236   I felt self-conscious around others.
   hitop258   I found it difficult to speak up in front of others.


   hitop1   I felt shy around other people.
   hitop17   I was uncomfortable meeting new people.
   hitop117   I felt socially awkward.
   hitop124   I avoided situations in which others were likely to watch me.
